In [1]:
pip install mistralai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.2/373.2 kB 20.8 MB/s eta 0:00:00


In [2]:
API_KEY='rrENEIP4CNCLjIyGyG9YGbgGXgMvcWPo'


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
import re
import base64
from mistralai import Mistral


def encode_image(image_location):
    with open(image_location, 'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def extract_details(ocr_text):
    # --- Normalize all markdown-style formatting and keyword variants ---
    normalized_text = re.sub(r"[*#]+", "", ocr_text)  # remove markdown asterisks and hashtags

    # Normalize variations of 'SHIP TO'
    normalized_text = re.sub(r"(SHIP[\s_]*TO)[:\-]?", "SHIP TO", normalized_text, flags=re.IGNORECASE)

    # --- Extract SHIP TO block ---
    ship_to_match = re.search(
        r"SHIP TO\s*(.*?)(?=\n(?:UPS|TRACKING|1Z|TILLING|COD|BILLING|REFERENCE|\Z))",
        normalized_text, re.IGNORECASE | re.DOTALL
    )
    ship_to_block = ship_to_match.group(1).strip() if ship_to_match else "Not found"

    # --- Extract individual lines from SHIP TO block ---
    ship_to_lines = [line.strip() for line in ship_to_block.splitlines() if line.strip()]
    name = ship_to_lines[0] if len(ship_to_lines) > 0 else "Not found"
    phone = ship_to_lines[1] if len(ship_to_lines) > 1 else "Not found"
    address_lines = ship_to_lines[2:] if len(ship_to_lines) > 2 else []
    address = ", ".join(address_lines)

    # --- Extract tracking number ---
    tracking_match = re.search(
        r"\b1Z[\s\dA-Z]{10,}\b",  # Match 1Z and at least ~16 alphanumeric characters
        normalized_text, re.IGNORECASE
    )
    tracking_number = (
        tracking_match.group(0).replace(" ", "").upper() if tracking_match else "Not found"
    )

    return name, phone, address, tracking_number



In [9]:
import os
API_KEY='rrENEIP4CNCLjIyGyG9YGbgGXgMvcWPo'

image_folder = '/content/drive/MyDrive/sample_imgs/'
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

data = []

for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    base64image = encode_image(image_path)

    ocr_response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "image_url",
            "image_url": f"data:image/jpg;base64,{base64image}"
        }
    )

    ocr_text = ocr_response.pages[0].markdown
    name, phone, address, tracking_number = extract_details(ocr_text)

    data.append({
        "Name": name,
        "Phone": phone,
        "Address": address,
        "Tracking Number": tracking_number
    })
    print(image_path)
    print("NAME:", name)
    print("PHONE:", phone)
    print("ADDRESS:", address)
    print("TRACKING NUMBER:", tracking_number)


/content/drive/MyDrive/sample_imgs/L16_aug14.jpg
NAME: JOHN SMITH
PHONE: 123 MAIN STREET
ADDRESS: GREENWICH CT 06830, ---, CT 069 9-01
TRACKING NUMBER: Not found
/content/drive/MyDrive/sample_imgs/L1_aug25.jpg
NAME: MATTHEW TOBIA
PHONE: 3154311352
ADDRESS: 5001 BRITTONFIELD PKWY, EAST SYRACUSE NY 13057, NY 132 9-01
TRACKING NUMBER: 1Z8402630298085320

BILLING
/content/drive/MyDrive/sample_imgs/L1_aug2.jpg
NAME: MATTHEW TOBIA
PHONE: 3154311352
ADDRESS: 5001 BRITTONFIELD PKWY, EAST SYRACUSE NY 13057, DPT, NY 1329-01
TRACKING NUMBER: Not found
/content/drive/MyDrive/sample_imgs/L1_aug3.jpg
NAME: MATTHEW TOBIA
PHONE: 3154311352
ADDRESS: 5001 BRITTONFIELD PKWY, EAST SYRACUSE NY 13057, DIVERSITY, NY 1329-01
TRACKING NUMBER: 1Z8402630298085320

BILLING
/content/drive/MyDrive/sample_imgs/L3_aug3.jpg
NAME: Not found
PHONE: Not found
ADDRESS: 
TRACKING NUMBER: Not found
/content/drive/MyDrive/sample_imgs/L4_aug6.jpg
NAME: LANE MARTIN
PHONE: 8801986582197
ADDRESS: BOLIVERDESIGNS, 190 ABC ST., DHA